In [28]:
include("../cliff7.jl")
using PyPlot
using Profile

In [40]:
function cc_adam_gate!(state, b, i, j, p)
    ms = size(state, 1)==size(state, 2) ? fps_measurement! : measurement!
    cliff = random_clifford(2b)
    rg = union(b*i-b+1:b*i, b*j-b+1:b*j)
    clifford_action!(cliff, state, rg)
    if rand() < p
        obs = (0, Bool[1, 0])
        for k in b*i-b+1:b*i
            ms(state, obs, [k])
        end
    end
    if rand() < p
        obs = (0, Bool[1, 0])
        for k in b*j-b+1:b*j
            ms(state, obs, [k])
        end
    end
    return nothing
end

function cc_ee_sim(n, b, cut, p_list, n_step, step_size, burnin, n_average, periodic)
    result = zeros(length(p_list), n_step)
    for (k, p) in enumerate(p_list)
        for _ in 1:n_average
            state = all_up(n*b)
            for _ in 1:burnin
                for ini in [1, 2]
                    for i in ini : 2 : (periodic ? n : n-1)
                        j = i==n ? 1 : i+1
                        cc_adam_gate!(state, b, i, j, p)
                    end
                end
            end
            for t in 1:n_step
#                 result[k, t] += left_ee_on_all_cuts(state)[b*cut+1]
                for _ in 1:step_size
                    for ini in [1, 2]
                        for i in ini : 2 : (periodic ? n : n-1)
                            j = i==n ? 1 : i+1
                            cc_adam_gate!(state, b, i, j, p)
                        end
                    end
                end
            end
        end
    end
    result ./= n_average
    return result
end

cc_ee_sim (generic function with 1 method)

In [37]:
n = 128
b = 2
cut = div(n, 2)
p_list = [0.2]
n_step = 128
step_size = 1
burnin = 0
n_average = 100
periodic = true
nn = 30

30

In [38]:
Profile.clear()
Profile.init(Int(1e7), 0.01)
@profile cc_ee_sim(n, b, cut, p_list, n_step, step_size, burnin, n_average, periodic)

1×128 Array{Float64,2}:
 0.0  2.52  4.62  6.26  8.03  9.61  11.07  …  34.8  34.79  34.85  34.7  34.77

In [39]:
Profile.print(mincount=5000)

Overhead ╎ [+additional indent] Count File:Line; Function
    ╎134002 @Base\task.jl:356; (::IJulia.var"#15#18")()
    ╎ 134002 ...ia\src\eventloop.jl:8; eventloop(::ZMQ.Socket)
    ╎  134002 @Base\essentials.jl:709; invokelatest
    ╎   134002 @Base\essentials.jl:710; #invokelatest#1
    ╎    134002 ...xecute_request.jl:67; execute_request(::ZMQ.Socket...
    ╎     134002 ...oftGlobalScope.jl:65; softscope_include_string(::...
  38╎    ╎ 134002 @Base\loading.jl:1091; include_string(::Function...
    ╎    ╎  20233  In[29]:35; cc_ee_sim(::Int64, ::Int6...
    ╎    ╎   19478  ...fford\cliff7.jl:376; left_ee_on_all_cuts(::Sta...
    ╎    ╎    19315  ...inary_linalg.jl:194; binary_all_vertical_cut_...
    ╎    ╎     19296  ...inary_linalg.jl:179; binary_all_vertical_cut_...
  29╎    ╎    ╎ 17406  ...nary_linalg.jl:35; binary_uppertrianglize...
    ╎    ╎    ╎  7348   ...tractarray.jl:1060; getindex
  16╎    ╎    ╎   7348   ...imensional.jl:743; _getindex
    ╎    ╎    ╎    6072   ...imensio